Imports

In [ ]:
#import time

#import gpu_imports
import numpy as np
import pandas as pd

#from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
#from sklearn.model_selection import train_test_split
#from sklearn.preprocessing import LabelEncoder
#import seaborn as sns
# import cv2
# import matplotlib.pyplot as plt

from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image as image_utils
#from tensorflow.keras.applications.imagenet_utils import preprocess_input
from keras.optimizers import RMSprop, SGD, Adam
#from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

#import sys

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Activation, Dense, Flatten, Dropout, ZeroPadding2D
from keras.layers import LeakyReLU

In [ ]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


In [ ]:
!gdown --id 1tj1pgxn8NzL9jCX3Bkh_4CczVIgUqHYV

Downloading...
From: https://drive.google.com/uc?id=1tj1pgxn8NzL9jCX3Bkh_4CczVIgUqHYV
To: /content/Malaria Data(with impurities).zip
337MB [00:01, 242MB/s]


In [ ]:
from zipfile import ZipFile
file_name="Malaria Data(with impurities).zip"

with ZipFile(file_name, 'r') as zip:
  zip.extractall()
  print('Done')

Done


Load data into x and y and randomize them

In [ ]:
#Malaria Data(with impurities)\Parasitized

malaria_images_path = './Malaria Data(with impurities)/Train' # Dataset contains folders Parasitized and Uninfected
Train_parasitized_path = malaria_images_path + '/Parasitized/'
Train_uninfected_path = malaria_images_path + '/Uninfected/'
# uninfected_path =  malaria_images_path + '/Uninfected'








In [ ]:
import os
parasitized_folder_T = os.listdir(Train_parasitized_path)
uninfected_folder_T = os.listdir(Train_uninfected_path)

In [ ]:
X_train = []
# Y = []
dim = (128, 128)
# count = 0

In [ ]:
for image in parasitized_folder_T:
    try:
        image = image_utils.load_img(Train_parasitized_path + image, color_mode="rgb", target_size=dim)
        #image = cv2.resize(image, dim)
        image = image_utils.img_to_array(image)
        X_train.append(image)
        #Y.append('Infected')
    except:
        continue

In [ ]:
for image in uninfected_folder_T:
    try:
        image = image_utils.load_img(Train_uninfected_path + image, color_mode="rgb", target_size=dim)
        #image = cv2.resize(image, dim)
        image = image_utils.img_to_array(image)
        X_train.append(image)
        #Y.append('Uninfected')
    except:
        continue

Preprocess training data
1. Resize
2. Normalize
3. change images into arrays


split the data into train, validation, and test samples

In [ ]:
# train_X, test_X, train_Y, test_Y = train_test_split(X, Y.astype(np.int8), 
#                                                                       test_size=0.3, random_state=42)
# train_X, val_X, train_Y, val_Y = train_test_split(X, Y.astype(np.int8), 
#                                                                     test_size=0.1, random_state=42)

Augment the data

In [ ]:

datagen = ImageDataGenerator(rotation_range=20,
                                  shear_range = 0.1,
                                  zoom_range = 0.05,
                                  #rescale=1.0/255.0,
                                  horizontal_flip=True,
                                  featurewise_center=True, 
                                  featurewise_std_normalization=True,
                             
                                  width_shift_range=0.05,
                                  height_shift_range=0.05)

datagen.fit(X_train)

#validation_datagen = ImageDataGenerator(rescale = 1./255)

#validation_datagen.fit(val_X)


In [ ]:
# load and iterate training dataset
train_it = datagen.flow_from_directory('Malaria Data(with impurities)/Train/', 
                                       target_size=(128,128), 
                                       color_mode='rgb', 
                                       class_mode='binary', 
                                       shuffle=True,
                                       seed=42,
                                       batch_size=32)
# load and iterate validation dataset
valid_it = datagen.flow_from_directory('Malaria Data(with impurities)/Valid/', 
                                      target_size=(128,128), 
                                      color_mode='rgb', 
                                      class_mode='binary', 
                                      shuffle=True,
                                       seed=20,
                                      batch_size=32)

test_it = datagen.flow_from_directory('Malaria Data(with impurities)/Test/', 
                                      target_size=(128,128), 
                                      color_mode='rgb',
                                      shuffle=True,
                                       seed=20, 
                                      class_mode='binary', 
                                      batch_size=32)

Found 15720 images belonging to 2 classes.
Found 5239 images belonging to 2 classes.
Found 5240 images belonging to 2 classes.


Make the model,

In [ ]:
leaky_relu_alpha = 0.1

model = Sequential()

# 1st Conv Layer
model.add(Conv2D(64, (3,3), strides=(1, 1), padding='same', input_shape=(128, 128, 3)))
model.add(LeakyReLU(alpha=leaky_relu_alpha))
model.add(BatchNormalization())

# 2nd Conv Layer
model.add(Conv2D(64, (3,3), strides=(1, 1), padding='same', input_shape=(128, 128, 3)))
model.add(LeakyReLU(alpha=leaky_relu_alpha))
model.add(BatchNormalization())

# Max Pooling with Dropout Layer
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.2))

# 3rd Conv Layer
model.add(Conv2D(128, (3,3), strides=(1, 1), padding='same'))
model.add(LeakyReLU(alpha=leaky_relu_alpha))
model.add(BatchNormalization())

# 4th Conv Layer
model.add(Conv2D(128, (3,3), strides=(1, 1), padding='same'))
model.add(LeakyReLU(alpha=leaky_relu_alpha))
model.add(BatchNormalization())

# Max Pooling with Dropout Layer
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.2))

# 5th Conv Layer
model.add(Conv2D(256, (3,3), strides=(1, 1), padding='same'))
model.add(LeakyReLU(alpha=leaky_relu_alpha))
model.add(BatchNormalization())

# 6th Conv Layer
model.add(Conv2D(256, (3,3), strides=(1, 1), padding='same'))
model.add(LeakyReLU(alpha=leaky_relu_alpha))
model.add(BatchNormalization())

# Max Pooling with Dropout Layer
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.2))

# 1st FC Dense Layer
model.add(Flatten())
model.add(Dense(256))
model.add(LeakyReLU(alpha=leaky_relu_alpha))
model.add(BatchNormalization())
model.add(Dropout(0.3))

# 2nd FC Dense Layer
model.add(Dense(256))
model.add(LeakyReLU(alpha=leaky_relu_alpha))
model.add(BatchNormalization())
model.add(Dropout(0.3))

# Final Dense Layer
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 128, 128, 64)      1792      
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 128, 128, 64)      0         
_________________________________________________________________
batch_normalization (BatchNo (None, 128, 128, 64)      256       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 128, 64)      36928     
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 128, 128, 64)      0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 128, 128, 64)      256       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 64, 64, 64)        0

Optimize

Compile

In [ ]:
from tensorflow import keras
from keras.optimizers import RMSprop, SGD, Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

checkpoint = ModelCheckpoint('/Saved_Model/Malaria_detection.h5',
                            monitor = 'val_loss',
                            mode = 'min',
                            save_best_only = True,
                            verbose = 1)

earlystop = EarlyStopping(monitor = 'val_loss',
                         min_delta = 0.0001,
                         patience = 20,
                         verbose = 1,
                         restore_best_weights = True)

reduce_lr = ReduceLROnPlateau(monitor = 'val_loss',
                             factor = 0.01,
                             patience = 20,
                             verbose = 1,
                             min_delta = 0.00001)

callbacks = [checkpoint, earlystop, reduce_lr]
model.compile(loss = 'binary_crossentropy',
             optimizer = Adam(learning_rate=0.0001, decay = 0.001 / 64),
             metrics=[keras.metrics.BinaryAccuracy()])

fit the model

In [ ]:
batch_size=64
steps_training=int(15722/batch_size)
steps_validation=int(5239/batch_size)
model.fit(train_it, 
          epochs = 50, 
          batch_size = batch_size,
          steps_per_epoch=steps_training,
          validation_data = valid_it,
          validation_steps=steps_validation)

Epoch 1/5
491/491 [==============================] - 109s 223ms/step - loss: 0.0253 - binary_accuracy: 0.9870 - val_loss: 0.1582 - val_binary_accuracy: 0.9869
Epoch 2/5
491/491 [==============================] - 109s 222ms/step - loss: 0.0250 - binary_accuracy: 0.9869 - val_loss: 0.1527 - val_binary_accuracy: 0.9869
Epoch 3/5
491/491 [==============================] - 108s 221ms/step - loss: 0.0261 - binary_accuracy: 0.9868 - val_loss: 0.1597 - val_binary_accuracy: 0.9868
Epoch 4/5
491/491 [==============================] - 109s 222ms/step - loss: 0.0241 - binary_accuracy: 0.9868 - val_loss: 0.1358 - val_binary_accuracy: 0.9867
Epoch 5/5
491/491 [==============================] - 109s 221ms/step - loss: 0.0244 - binary_accuracy: 0.9866 - val_loss: 0.0915 - val_binary_accuracy: 0.9866


In [ ]:
#test_accuracy = model.evaluate(test_it, verbose = 1)

In [ ]:
#model.save('malaria_detection_model_2')

INFO:tensorflow:Assets written to: malaria_detection_model_2/assets
